# Fase 4: Análisis Avanzado y Flujo de Diálogo

Este notebook cubre las Actividades 4 a 7 de la Ruta de Aprendizaje Autónomo 3:
4.  **Generación de Vectores**: Word2Vec vs S-BERT.
5.  **Análisis Comparativo**: Clustering de tópicos y Analogías.
6.  **Flujo de Diálogo**: Integración con LLM (Ollama).
7.  **Pruebas**: Simulación de consultas.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time

# Embeddings & Clustering
from gensim.models import Word2Vec
from sentence_transformers import SentenceTransformer
import umap.umap_ as umap
import hdbscan

# LangChain & LLM
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

## 1. Carga de Datos

In [ ]:
data_path = Path("../data/processed_corpus.csv")
df = pd.read_csv(data_path)
df = df.dropna(subset=['clean_text'])

print(f"Corpus Total: {len(df)} documentos")
texts = df['clean_text'].astype(str).tolist()

## 2. Actividad 4: Generación de Vectores (Word2Vec vs S-BERT)

Entrenaremos un modelo `Word2Vec` desde cero (captura relaciones sintácticas específicas del corpus) y usaremos `S-BERT` pre-entrenado (captura semántica profunda).

In [ ]:
# === A. Word2Vec (Entrenamiento Local) ===
print("Entrenando Word2Vec...")
tokenized_texts = [t.split() for t in texts]
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=2, workers=4)
print("Word2Vec entrenado.")

# === B. S-BERT (Pre-entrenado) ===
print("Cargando S-BERT (all-MiniLM-L6-v2)...")
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_sbert = sbert_model.encode(texts, show_progress_bar=True)

## 3. Actividad 5: Análisis Comparativo (Analogías y Clustering)

### 3.1 Analogías con Word2Vec
Intentamos resolver relaciones semánticas tipo: *Rey - Hombre + Mujer = Reina*.

In [ ]:
def test_analogy(w1, w2, w3):
    try:
        result = w2v_model.wv.most_similar(positive=[w1, w3], negative=[w2], topn=1)
        print(f"{w1} - {w2} + {w3} = {result[0][0]} ({result[0][1]:.2f})")
    except KeyError as e:
        print(f"Error: Palabras no encontradas en vocabulario ({e})")

# Prueba con términos hipotéticos del dominio urbano (ajustar según vocabulario real)
print("--- Pruebas de Analogía ---")
# Ejemplo: Si 'calle' es a 'asfalto', 'acera' es a...?
test_analogy('calle', 'coche', 'acera') 
# Ejemplo: 'suciedad' - 'basura' ...
test_analogy('ruido', 'trafico', 'basura')

### 3.2 Discovery de Tópicos (Clustering con UMAP + HDBSCAN)
Usamos los embeddings de S-BERT para encontrar grupos semánticos automáticamente.

In [ ]:
# 1. Reducción de Dimensionalidad (UMAP)
print("Reduciendo dimensiones con UMAP...")
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=2, 
                            metric='cosine').fit_transform(embeddings_sbert)

# 2. Clustering (HDBSCAN)
print("Clusterizando con HDBSCAN...")
clusterer = hdbscan.HDBSCAN(min_cluster_size=15,
                            metric='euclidean',
                            cluster_selection_method='eom')
cluster_labels = clusterer.fit_predict(umap_embeddings)

df['cluster'] = cluster_labels
n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
print(f"Clusters encontrados: {n_clusters} (Ruido asignado a -1)")

In [ ]:
# Visualización
plt.figure(figsize=(10, 8))
plt.scatter(umap_embeddings[:, 0], umap_embeddings[:, 1], c=cluster_labels, cmap='Spectral', s=5)
plt.colorbar(label='Cluster ID')
plt.title('Proyección UMAP de Quejas Ciudadanas (S-BERT)')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.show()

In [ ]:
# Inspección de Clusters (Top palabras por cluster)
# Una forma simple es ver los textos más cercanos al centroide o textos aleatorios
for i in range(n_clusters):
    print(f"\n--- Cluster {i} ---")
    sample_docs = df[df['cluster'] == i]['clean_text'].sample(5).values
    for doc in sample_docs:
        print(f"- {doc[:80]}...")

## 4. Actividad 6 y 7: Flujo de Diálogo & RAG (LangChain + Ollama)

Simularemos un asistente inteligente que recupera documentos relevantes (RAG) y genera respuestas usando `gemma3:1b`.

In [ ]:
# Configurar LLM Local
llm = ChatOllama(model="gemma3:1b", temperature=0.3)

# --- Simple Retriever (basado en Coseno con S-BERT) ---
def retrieve_documents(query, k=3):
    query_vec = sbert_model.encode([query])
    sims = cosine_similarity(query_vec, embeddings_sbert)[0]
    # Top k indices
    top_indices = sims.argsort()[-k:][::-1]
    return df.iloc[top_indices]['clean_text'].tolist()

# --- Chain Definition ---
prompt_template = """
Eres un asistente de IA para una plataforma de gestión urbana. 
Usa el siguiente contexto (quejas reales de ciudadanos) para responder a la pregunta del usuario. 
Si no sabes la respuesta basada en el contexto, dilo claramente.

Contexto:
{context}

Pregunta Ciudadana: {question}

Respuesta Útil y Empática:
"""
prompt = PromptTemplate.from_template(prompt_template)

def run_rag(question):
    start_time = time.time()
    
    # 1. Retrieve
    docs = retrieve_documents(question)
    context_str = "\n".join([f"- {d}" for d in docs])
    
    # 2. Generate
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"context": context_str, "question": question})
    
    latency = time.time() - start_time
    return response, context_str, latency

In [ ]:
# --- Simulación de Consultas (Actividad 7) ---
test_queries = [
    "¿Cuáles son las quejas más comunes sobre el tráfico?",
    "Hay mucha basura en las calles, ¿qué dice la gente?",
    "¿Es segura la zona del centro por la noche?"
]

for q in test_queries:
    print(f"\n{'='*50}")
    print(f"Pregunta: {q}")
    ans, ctx, lat = run_rag(q)
    print(f"Latencia: {lat:.2f}s")
    print(f"Contexto Recuperado (Snippet): {ctx[:200]}...")
    print(f"\nRespuesta IA:\n{ans}")